In [1]:
from getpass import getpass
import requests
import pandas as pd
import math
from serlo.mysql import db

MAILCHIMP_API_KEY = getpass("MAILCHIMP_API_KEY")
dc = MAILCHIMP_API_KEY.split("-")[1]

mailchimp_api_url = f"https://{dc}.api.mailchimp.com/3.0/lists/a7bb2bbc4f/members"

MAILCHIMP_API_KEY········


In [2]:
# Hier das Datum eintragen, ab dem Subscriptions gesucht werden sollen.

date = "2020-03-01T00:00:00+00:00"

In [3]:
# TODO: Entscheiden, ob die Description hier rausgenommen wird, da es ein anderer Anwendungsfall ist
df = pd.read_sql(
    """
    Select
        user.id as serlo_id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""",
    db,
)

df.fillna(0, inplace=True)

df["date"] = df["date"].map(pd.to_datetime)
inactive_users = df[
    (df["edits"] + df["reviews"] + df["taxonomy"] == 0)
    & (df["description"] != "NULL")
    & (df["description"] != "")
    & (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
inactive_users.sort_values(["date"], inplace=True)
inactive_users

compact_inactive_users = inactive_users[["serlo_id", "username"]]

/tmp/ipykernel_20865/2075254977.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [4]:
subscribers = requests.get(
    mailchimp_api_url,
    params={
        "apikey": MAILCHIMP_API_KEY,
        "status": "subscribed",
        "fields": "total_items",
    },
)
total_subs = subscribers.json()["total_items"]
print(f"Anzahl der Newsletter-Subscriptions: {total_subs}")

Anzahl der Newsletter-Subscriptions: 8189


In [5]:
subscribers = []
for i in range(math.ceil(total_subs / 1000)):
    subscribers.extend(
        requests.get(
            mailchimp_api_url,
            params={
                "apikey": MAILCHIMP_API_KEY,
                "status": "subscribed",
                "since_timestamp_opt": date,
                "offset": (i * 1000),
                "count": 1000,
            },
        ).json()["members"]
    )

In [6]:
has_not_opened_newsletter = list(
    filter(lambda subscriber: (subscriber["stats"]["avg_open_rate"] == 0), subscribers)
)

print(
    f"Anzahl Subscriber, die keinen Newsletter geöffnet haben: {len(has_not_opened_newsletter)}"
)

Anzahl Subscriber, die keinen Newsletter geöffnet haben: 3215


In [7]:
merged_data = {}

for subscriber in has_not_opened_newsletter:
    unique_key = subscriber["merge_fields"]["UNAME"]
    merged_data[unique_key] = subscriber

for index, user in compact_inactive_users.iterrows():
    #     print(user)
    unique_key = user["username"]
    if unique_key in merged_data:
        merged_data[unique_key].update(user)

linked_account = list(
    value
    for value in merged_data.values()
    if all(key in value for key in ("serlo_id", "id"))
)
df = pd.json_normalize(linked_account)
df.rename(columns={"id": "mailchimp_id"}, inplace=True)
len(df)

41

In [8]:
"""
Das ist Code, den ich zum Testen verwende.

MY_TEST_MAILCHIMP_API_KEY = ""
dc2 = MY_TEST_MAILCHIMP_API_KEY.split('-')[1]
act_url = f"https://{dc2}.api.mailchimp.com/3.0/lists/66709f13c5/members"
activity = requests.get(act_url, params = {"apikey": MY_TEST_MAILCHIMP_API_KEY})
activity.json()
"""

'\nDas ist Code, den ich zum Testen verwende.\n\nMY_TEST_MAILCHIMP_API_KEY = ""\ndc2 = MY_TEST_MAILCHIMP_API_KEY.split(\'-\')[1]\nact_url = f"https://{dc2}.api.mailchimp.com/3.0/lists/66709f13c5/members"\nactivity = requests.get(act_url, params = {"apikey": MY_TEST_MAILCHIMP_API_KEY})\nactivity.json()\n'

In [9]:
"""
members = activity.json()["members"]
test = filter(lambda member: (member["stats"]["avg_open_rate"] != 0) & (member["status"] == "subscribed"), members)
list(test)
"""

'\nmembers = activity.json()["members"]\ntest = filter(lambda member: (member["stats"]["avg_open_rate"] != 0) & (member["status"] == "subscribed"), members)\nlist(test)\n'

In [10]:
def is_possible_fake_account(subscriber):
    return subscriber['merge_fields']["UNAME"] in [user['username'] for index, user in compact_inactive_users.iterrows()]

filtered = [account for account in linked_account if is_possible_fake_account(account)]
print(
    f"Anzahl Subscriber, die keinen Newsletter geöffnet haben und keine Aktivität auf serlo.org haben: {len(filtered)}"
)

print(f"Ids: {[user['id'] for user in filtered]}")

Anzahl Subscriber, die keinen Newsletter geöffnet haben und keine Aktivität auf serlo.org haben: 41
Ids: ['691f6292ad2656478a067149db04e7da', 'c8d54ff1aad00299498ab075125f6295', '951fec9d40ceb5a160af31de6a403ae8', '821a2ca232ae87b80ae539e0ccd1e59b', '2f15929b9959851fb67a4ffc774a2849', '77d6dd0226118c71886626103c7ab326', '1f65ee1fcb0af5526d44347837c725e0', '80433639f86df9f7ae20f4a452295875', '7e16779e687bd00b1899e85cd9eeaaf3', '42a75916c4319f6d0ec9b66d87031098', 'a2d270b1839c9a32862ecdf223d6513d', '86af5b350ff9cdf802ac717ae552d0be', '98bd23e4668b78a4992ac4c9beacd7ee', 'ab2f597b9df7c63c84047b84e4d81c9c', '1b9530ab2307d45dfe7a5220a5b3dd4a', 'f9da855441a49a629d820703399527ef', '31be3e67edaa25a2b773b03a4f00676e', '73e48bb2a3e5e87729584f0a73eec3cf', '4633c1253d613b59ffee770123865a9c', 'e50156013a0add6cf42bf09963660603', 'f2b4c3268de285ffb21402aab5567de0', '401997bd486191814894905096222fc1', 'e10b1d5c16d76d8f6fe42cfc7dd2eb2f', '85620a47f6d276b61ff1ba73cadabe82', '478c85627404732927a7616e90431

In [11]:
### Das sind alles Subscriber, von denen wir auch den Account auf serlo.org kennen.
### You can check the profile of the users here: serlo.org/{id}
### Die Liste der Ids samt Usernamen, die von Serlo gelöscht werden sollen, bitte an moritz@serlo.org schicken oder,
### falls nicht verfügbar, an den Admin eures Vertrauens ;)
### Falls nur die Subscription gelöscht werden soll, bitte in den nächsten Abschnitt schauen.

In [12]:
# TODO: Which information is relevant?
not_linked_subscribers = list(
    filter(
        lambda member: not member["merge_fields"]["UNAME"], has_not_opened_newsletter
    )
)
# print(
#     pd.json_normalize(not_linked_subscribers)[
#         ["id", "email_address", "full_name", "timestamp_signup"]
#     ]
# )

In [13]:
# Fill in here the mailchimp id (important: not the serlo id!) of the subscribers 
# you want to remove from the newsletter list like that [id1, id2, id3, ...]
ids_to_remove = []

In [14]:
def delete_url(subscriber_hash):
    return f"/lists/a7bb2bbc4f/members/{subscriber_hash}/actions/delete-permanent"

# Uncomment if we are supposed to delete accounts automatically
# for subscriber_hash in ids_to_remove:
#     requests.post(delete_url(subscriber_hash), key: MAILCHIMP_API_KEY)